# **Сравнительный анализ  нейросетевых архитектур при решении задачи многоклассовой классификации**

Долгое время CNN и RNN архитектуры были конкурирующими. Необходимо взять датасет отзывов за лето и сравнить архитектуры при решении задачи многоклассовой классификации. Построить сверточные архитектуры, различные рекуррентные архитектуры, а также совместные архитектуры CNN -> RNN или (RNN -> CNN).

# Подготовка данных

Установим необходимые библиотеки и загрузим данные.

In [ ]:
!pip install stop_words
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=8773999908014af478f46f9ecbd6714d09d8d7e9da69b452978e895ff3e422df
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 915.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5e22966c001e3a516371e50d2739bde43343332088c3975a64ccdc14920a1465
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import keras
import nltk
import numpy as np
import pandas as pd
import re
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Activation, Bidirectional, Conv1D, Dense, Dropout, Embedding, GlobalMaxPool1D, GRU, Input, LSTM, Masking, SimpleRNN
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from stop_words import get_stop_words
from string import punctuation
from tqdm import tqdm
nltk.download("punkt")
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_excel('отзывы за лето.xls')

In [ ]:
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


Датасет состоит из 3 столбоцв: 	Rating – рейтинг (целевая переменная), 	Content – отзыв, 	Date –дата.  

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


Датасет содержит 20659 строк, пропусков нет.

Посмотрим на значения целевой переменной.


In [ ]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Целевая переменная содержит 5 значений от 1 до 5, при этом классы распределены неравномерно: преобладает класс 5, меньше всего представлен класс 2. Для данных с отзывами - это нормальная ситуация, рейтинговая система по природе несбалансирована. Устранение дисбаланса уничтожит эту бизнес-реалистичную картину. Чтобы учитывать дисбаланс в качестве метрики будем использовать F1.

Сделаем предобработку отзывов: приведем слова в к нижнему регистру, удалим знаки препинания и небуквенные символы, приведем слова к нормальной форме.

In [ ]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = re.sub(patterns, ' ', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [ ]:
df['Content'] = df['Content'].progress_apply(preprocess_text)

100%|██████████| 20659/20659 [00:14<00:00, 1475.01it/s]


Создадим корпус слов.

In [ ]:
train_corpus = " ".join(df["Content"])
tokens = word_tokenize(train_corpus)

In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

Создадим переменные, которые будут использоваться в качестве гиперпараметров сети.

In [ ]:
MAX_WORDS = 2000 # максимальное количество уникальных слов
MAX_LEN = 40 # максимальное количество токенов в документе
NUM_CLASSES = 5

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

Вычислим частоту токенов  и возьмем наиболее частотные.

In [ ]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]

Выведем для примера 10 самых частотных токенов.

In [ ]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

Из массива самых частотных слов делаем словарь.

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

Далее создадим функцию text_to_sequence для перевода текста в последовательность токенов: произведем токенизацию, проверим нет ли специальных символов. Если токен есть в топ 2000 - добавляем его в результирующий набор данных. Пустые символы заполним 0.

In [ ]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

Разобьем данные на тренировочные и тестовые и применим к ним преобразования.

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [ ]:
x_train = np.asarray([text_to_sequence(text, MAX_LEN) for text in df_train['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, MAX_LEN) for text in df_test['Content']], dtype=np.int32)

In [ ]:
x_train.shape

(13841, 40)

In [ ]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,
       179], dtype=int32)

К порядковой целевой переменной применим LabelEncoder для преобразования категориальных меток в числовые значения.

In [ ]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating'])
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

Далее преобразуем вектор класса в двоичную классную матрицу для дальнейшего использования с функцией categorical_crossentropy, которая ожидает, что метки следуют бинарному кодированию.

In [ ]:
y_train = keras.utils.to_categorical(train_enc_labels, num_classes=NUM_CLASSES)
y_test = keras.utils.to_categorical(test_enc_labels, num_classes=NUM_CLASSES)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

# Обучение

### CNN

Попробуем использовать архитектуру со сверточным слоем, так как свертки могут успешно обрабатывать тексты, потому что он эффективно выявляют локальные паттерны и n-граммы в последовательностях слов, подобно тому как фильтры в изображениях обнаруживают характерные признаки.

Создадим модель с одним сверточным слоем: сначала расположим слой эмбеддинга для перевода токенов в векторное представление, затем сверточный слой с функцией активации "relu" для поддержания потока градиента, затем GlobalMaxPool1D для сохранения наиболее важных признаков, далее полносвязный слой с "relu" и полносвязный слой с 'softmax для агрегирования информации , полученной на предыдущих этапах обработки и вывода финального предсказания.

In [ ]:
model_CNN = Sequential()
model_CNN.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN))
model_CNN.add(Conv1D(128, 3))
model_CNN.add(Activation("relu"))
model_CNN.add(GlobalMaxPool1D())
model_CNN.add(Dense(10))
model_CNN.add(Activation("relu"))
model_CNN.add(Dense(NUM_CLASSES))
model_CNN.add(Activation('softmax'))

Скомпилируем модель: в качестве функции потерь будем использовать категориальную кросс-энтропию, так как у нас задача многоклассовой классификации; зададим оптимизатор Adam, так как он хорошо показывает себя во многих задачах, сочетая преимущества AdaGrad и RMSProp; метрика – accuracy.

In [ ]:
model_CNN.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Для контроля переобучения будем использовать Early Stopping – обучение остановится, когда val_loss перестанет улучшаться.

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')
history = model_CNN.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 4s 115ms/step - loss: 1.2317 - accuracy: 0.6727 - val_loss: 1.0220 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 2s 93ms/step - loss: 0.8956 - accuracy: 0.7095 - val_loss: 0.8134 - val_accuracy: 0.7278
Epoch 3/20
25/25 [==============================] - 2s 93ms/step - loss: 0.7347 - accuracy: 0.7612 - val_loss: 0.7430 - val_accuracy: 0.7458
Epoch 4/20
25/25 [==============================] - 2s 92ms/step - loss: 0.6697 - accuracy: 0.7802 - val_loss: 0.7084 - val_accuracy: 0.7545
Epoch 5/20
25/25 [==============================] - 2s 92ms/step - loss: 0.6278 - accuracy: 0.7925 - val_loss: 0.6966 - val_accuracy: 0.7603
Epoch 6/20
25/25 [==============================] - 3s 112ms/step - loss: 0.5975 - accuracy: 0.8014 - val_loss: 0.6895 - val_accuracy: 0.7632
Epoch 7/20
25/25 [==============================] - 2s 93ms/step - loss: 0.5717 - accuracy: 0.8105 - val_loss: 0.6889 - val_accuracy: 0.7596
Epoch 8/20


Видим, что на тренировочном датасете loss снижается постепенно, и accuracy растет тоже постепенно. Но на тестовых данных loss  и accuracy к концу обучения ведут себя немного скачкообразно, что может указывать не переобучение.

In [ ]:
score_CNN = model_CNN.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_CNN[0])
print('Test f1_score:', score_CNN[1])

14/14 [==============================] - 0s 25ms/step - loss: 0.6555 - accuracy: 0.7765


Test loss: 0.6555193066596985
Test f1_score: 0.7764740586280823


Метрики на тестовом наборе получились не очень высокими. Попробуем другие архитектуры.  

### SimpleRNN

Попробуем использовать архитектуру со слоем SimpleRNN, так как он может хорошо справляться с текстами благодаря способности обрабатывать последовательности и сохранять контекст через скрытое состояние, что позволяет улавливать локальные зависимости между словами.

Создадим модель с одним рекуррентным слоем: сначала расположим слой эмбеддинга для перевода токенов в векторное представление, добавив параметр 'mask_zero=True', чтобы модель игнорировала нули в конце последовательности; затем слой Masking для того, чтобы сообщить слоям, обрабатывающим последовательность, что часть значений во входных данных отсутствует и должна быть пропущена при обработке данных; затем SimpleRNN, полносвязный слой с "relu" для добавления нелинейности обучению; слой Dropout для выключения части нейронов и предотвращения переобучения; и полносвязный слой с 'softmax для агрегирования информации, полученной на предыдущих этапах обработки и вывода финального предсказания.


In [ ]:
model_SimpleRNN = Sequential()
model_SimpleRNN.add(Embedding(input_dim=MAX_WORDS, input_length=MAX_LEN, output_dim=128, mask_zero=True))
model_SimpleRNN.add(Masking(mask_value=0.))
model_SimpleRNN.add(SimpleRNN(128))
model_SimpleRNN.add(Dense(10, activation='relu'))
model_SimpleRNN.add(Dropout(0.5))
model_SimpleRNN.add(Dense(NUM_CLASSES, activation='softmax'))

Скомпилируем модель с параметрами, использованными ранее, и выполним обучение.

In [ ]:
model_SimpleRNN.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_SimpleRNN.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 4s 130ms/step - loss: 1.2767 - accuracy: 0.5692 - val_loss: 0.9402 - val_accuracy: 0.7097
Epoch 2/20
25/25 [==============================] - 3s 119ms/step - loss: 1.1127 - accuracy: 0.6563 - val_loss: 0.8628 - val_accuracy: 0.7227
Epoch 3/20
25/25 [==============================] - 2s 95ms/step - loss: 1.0155 - accuracy: 0.7244 - val_loss: 0.8062 - val_accuracy: 0.7292
Epoch 4/20
25/25 [==============================] - 2s 95ms/step - loss: 0.9363 - accuracy: 0.7323 - val_loss: 0.7813 - val_accuracy: 0.7365
Epoch 5/20
25/25 [==============================] - 3s 116ms/step - loss: 0.8689 - accuracy: 0.7476 - val_loss: 0.7630 - val_accuracy: 0.7401
Epoch 6/20
25/25 [==============================] - 2s 95ms/step - loss: 0.8179 - accuracy: 0.7502 - val_loss: 0.7320 - val_accuracy: 0.7473
Epoch 7/20
25/25 [==============================] - 2s 95ms/step - loss: 0.7772 - accuracy: 0.7623 - val_loss: 0.7461 - val_accuracy: 0.7466


Видим, что на тренировочном датасете loss снижается постепенно, и accuracy растет тоже постепенно. На тестовых данных loss и accuracy ведут себя более стабильно по сравнению с предыдущей моделью.

In [ ]:
score_SimpleRNN = model_SimpleRNN.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_SimpleRNN[0])
print('Test f1_score:', score_SimpleRNN[1])

14/14 [==============================] - 0s 29ms/step - loss: 0.7100 - accuracy: 0.7606


Test loss: 0.7100391387939453
Test f1_score: 0.7606335878372192


Loss и f1_score  немного ухудшились. Попробуем другие виды архитектур.

### LSTM

Попробуем использовать архитектуру со слоем LSTM, так как он может успешно работать с текстами благодаря своей способности сохранять и управлять долгосрочными зависимостями в последовательностях через механизмы забывания, обновления и запоминания информации.

Создадим модель со слоем LSTM: сначала расположим слой эмбеддинга для перевода токенов в векторное представление, добавив параметр ‘mask_zero=True’; затем слой Masking, чтобы учитывать только релевантные элементы последовательности; затем слой LSTM, полносвязный слой с "relu" для добавления нелинейности обучению; слой Dropout для выключения части нейронов и предотвращения переобучения; и полносвязный слой с 'softmax для агрегирования информации, полученной на предыдущих этапах обработки и вывода финального предсказания.



In [ ]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(input_dim=MAX_WORDS, input_length=MAX_LEN, output_dim=128, mask_zero=True))
model_LSTM.add(Masking(mask_value=0.0))
model_LSTM.add(LSTM(64, recurrent_dropout=0.2))
model_LSTM.add(Dense(64, activation='relu'))
model_LSTM.add(Dropout(0.5))
model_LSTM.add(Dense(NUM_CLASSES, activation='softmax'))

Скомпилируем модель с параметрами, использованными ранее, и выполним обучение.

In [ ]:
model_LSTM.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_LSTM.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 8s 218ms/step - loss: 1.2461 - accuracy: 0.6561 - val_loss: 0.9145 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 5s 194ms/step - loss: 0.8876 - accuracy: 0.7109 - val_loss: 0.7834 - val_accuracy: 0.7292
Epoch 3/20
25/25 [==============================] - 5s 216ms/step - loss: 0.7662 - accuracy: 0.7472 - val_loss: 0.7244 - val_accuracy: 0.7386
Epoch 4/20
25/25 [==============================] - 5s 194ms/step - loss: 0.7011 - accuracy: 0.7622 - val_loss: 0.7012 - val_accuracy: 0.7458
Epoch 5/20
25/25 [==============================] - 5s 216ms/step - loss: 0.6589 - accuracy: 0.7762 - val_loss: 0.6943 - val_accuracy: 0.7487
Epoch 6/20
25/25 [==============================] - 5s 195ms/step - loss: 0.6307 - accuracy: 0.7852 - val_loss: 0.6910 - val_accuracy: 0.7552
Epoch 7/20
25/25 [==============================] - 5s 218ms/step - loss: 0.6062 - accuracy: 0.7950 - val_loss: 0.7057 - val_accuracy: 0.7531


Обучение происходит достаточно стабильно.

In [ ]:
score_LSTM = model_LSTM.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_LSTM[0])
print('Test f1_score:', score_LSTM[1])

14/14 [==============================] - 1s 50ms/step - loss: 0.6776 - accuracy: 0.7732


Test loss: 0.6775962710380554
Test f1_score: 0.7732473015785217


Loss лучше SimpleRNN, но немного хуже CNN. f1_score близок по значению к CNN.

### GRU

Попробуем использовать архитектуру со слоем GRU, так как он может быть эффективен для текстов, потому что его упрощённая архитектура с механизмами обновления и сброса эффективно улавливает долгосрочные зависимости, будучи при этом вычислительно менее затратной, чем LSTM. Архитектура будет состять из слоев Embedding, Masking, GRU, Dropout и Dense.

In [ ]:
model_GRU = Sequential()
model_GRU.add(Embedding(input_dim=MAX_WORDS, input_length=MAX_LEN, output_dim=128, mask_zero=True))
model_GRU.add(Masking(mask_value=0.0))
model_GRU.add(GRU(10, recurrent_dropout=0.2))
model_GRU.add(Dense(64, activation='relu'))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(NUM_CLASSES, activation='softmax'))

Скомпилируем модель с параметрами, использованными ранее, и выполним обучение.

In [ ]:
model_GRU.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_GRU.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 4s 89ms/step - loss: 1.5051 - accuracy: 0.6348 - val_loss: 1.2861 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 2s 84ms/step - loss: 1.0226 - accuracy: 0.7030 - val_loss: 0.8354 - val_accuracy: 0.7011
Epoch 3/20
25/25 [==============================] - 2s 97ms/step - loss: 0.8100 - accuracy: 0.7146 - val_loss: 0.7551 - val_accuracy: 0.7148
Epoch 4/20
25/25 [==============================] - 2s 67ms/step - loss: 0.7370 - accuracy: 0.7547 - val_loss: 0.7317 - val_accuracy: 0.7365
Epoch 5/20
25/25 [==============================] - 2s 63ms/step - loss: 0.6961 - accuracy: 0.7704 - val_loss: 0.7240 - val_accuracy: 0.7386
Epoch 6/20
25/25 [==============================] - 2s 78ms/step - loss: 0.6725 - accuracy: 0.7763 - val_loss: 0.7191 - val_accuracy: 0.7437
Epoch 7/20
25/25 [==============================] - 2s 67ms/step - loss: 0.6457 - accuracy: 0.7812 - val_loss: 0.7171 - val_accuracy: 0.7437
Epoch 8/20
25

Обучение происходит достаточно стабильно.

In [ ]:
score_GRU = model_GRU.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_GRU[0])
print('Test f1_score:', score_GRU[1])

14/14 [==============================] - 0s 15ms/step - loss: 0.6909 - accuracy: 0.7618


Test loss: 0.6908633708953857
Test f1_score: 0.7618069648742676


Получен второй по величине Loss среди экспериментов – чуть хуже, чем loss у CNN. f1_score чуть лучше, чем самый плохой полученный f1 у модели с SimpleRNN.



### Bidirectional SimpleRNN

Bidirectional SimpleRNN может быть удачным выбором для классификации текстов, потому что он обрабатывает последовательность в обоих направлениях - это позволяет учитывать контекст как от предыдущих, так и от последующих слов одновременно.

Создадим архитектуру со слоями Embedding, Dropout и Bidirectional SimpleRNN.


In [ ]:
model_BSimpleRNN = Sequential()
model_BSimpleRNN.add(Embedding(input_dim=MAX_WORDS, input_length=MAX_LEN, output_dim=128, mask_zero=True))
model_BSimpleRNN.add(Dropout(0.4))
model_BSimpleRNN.add(Bidirectional(SimpleRNN(64, dropout=0.05, recurrent_dropout=0.2)))
model_BSimpleRNN.add(Dense(NUM_CLASSES, activation='softmax'))

Скомпилируем модель с параметрами, использованными ранее, и выполним обучение.

In [ ]:
model_BSimpleRNN.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_BSimpleRNN.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 7s 182ms/step - loss: 1.3432 - accuracy: 0.6420 - val_loss: 1.0322 - val_accuracy: 0.7119
Epoch 2/20
25/25 [==============================] - 5s 188ms/step - loss: 0.8786 - accuracy: 0.7377 - val_loss: 0.8488 - val_accuracy: 0.7343
Epoch 3/20
25/25 [==============================] - 4s 167ms/step - loss: 0.7629 - accuracy: 0.7695 - val_loss: 0.8097 - val_accuracy: 0.7495
Epoch 4/20
25/25 [==============================] - 5s 195ms/step - loss: 0.7108 - accuracy: 0.7847 - val_loss: 0.7820 - val_accuracy: 0.7509
Epoch 5/20
25/25 [==============================] - 4s 170ms/step - loss: 0.6760 - accuracy: 0.7960 - val_loss: 0.7782 - val_accuracy: 0.7603
Epoch 6/20
25/25 [==============================] - 4s 169ms/step - loss: 0.6411 - accuracy: 0.8045 - val_loss: 0.7678 - val_accuracy: 0.7682
Epoch 7/20
25/25 [==============================] - 5s 192ms/step - loss: 0.6206 - accuracy: 0.8132 - val_loss: 0.7771 - val_accuracy: 0.7675


Обучение происходит достаточно стабильно.

In [ ]:
score_BSimpleRNN = model_BSimpleRNN.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_BSimpleRNN[0])
print('Test f1_score:', score_BSimpleRNN[1])

14/14 [==============================] - 0s 31ms/step - loss: 0.7388 - accuracy: 0.7800


Test loss: 0.7387878894805908
Test f1_score: 0.7799941301345825


На данный момент получен самый большой f1_score среди всех экспериментов. Loss также больше по сравнению с другими моделями, но значение является нормальным, учитывая сложность задачи.

Попробуем другие виды архитектур.

### CNN -> LSTM

Для улучшения качества обучения попробуем комбинацию слоев CNN -> LSTM, предполагая, что CNN эффективно извлечет локальные признаки и n-граммы, а LSTM обработает их как последовательность, улавливая долгосрочные зависимости и контекст. Создадим архитектуру из слоев: Embedding, CNN, Dropout, LSTM и Dense для финального результата.

In [ ]:
model_CNN_LSTM = Sequential()
model_CNN_LSTM.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN))
model_CNN_LSTM.add(Conv1D(128, 3))
model_CNN_LSTM.add(Activation("relu"))
model_CNN_LSTM.add(Dropout(0.2))
model_CNN_LSTM.add(LSTM(128))
model_CNN_LSTM.add(Dense(NUM_CLASSES, activation='softmax'))

Скомпилируем модель с параметрами, использованными ранее, и выполним обучение.

In [ ]:
model_CNN_LSTM.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_CNN_LSTM.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 14s 426ms/step - loss: 1.0518 - accuracy: 0.6774 - val_loss: 0.8462 - val_accuracy: 0.7018
Epoch 2/20
25/25 [==============================] - 11s 449ms/step - loss: 0.7700 - accuracy: 0.7412 - val_loss: 0.7466 - val_accuracy: 0.7379
Epoch 3/20
25/25 [==============================] - 9s 376ms/step - loss: 0.6733 - accuracy: 0.7657 - val_loss: 0.7015 - val_accuracy: 0.7487
Epoch 4/20
25/25 [==============================] - 9s 362ms/step - loss: 0.6121 - accuracy: 0.7854 - val_loss: 0.6899 - val_accuracy: 0.7603
Epoch 5/20
25/25 [==============================] - 9s 362ms/step - loss: 0.5737 - accuracy: 0.7999 - val_loss: 0.6911 - val_accuracy: 0.7625


Обучение происходит достаточно стабильно.

In [ ]:
score_CNN_LSTM = model_CNN_LSTM.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_CNN_LSTM[0])
print('Test f1_score:', score_CNN_LSTM[1])

14/14 [==============================] - 2s 107ms/step - loss: 0.6673 - accuracy: 0.7725


Test loss: 0.6673223376274109
Test f1_score: 0.7725139260292053


Мы получили второй по величине loss после CNN. f1_score сопоставим с Bidirectional_SimpleRNN, LSTM и CNN.

### LSTM -> CNN

Также попробуем комбинацию слоев LSTM -> CNN, предполагая, что LSTM сначала извлечет контекстуальные зависимости и структуру последовательности, а CNN затем выделяет наиболее значимые иерархические паттерны из этих обогащённых представлений. Создадим архитектуру со слоями Embedding, LSTM, Dropout, Conv1D и Dense для финального результата.

In [ ]:
model_LSTM_CNN=Sequential()
model_LSTM_CNN.add(Embedding(input_dim=MAX_WORDS, input_length=MAX_LEN, output_dim=128, mask_zero=True))
model_LSTM_CNN.add(LSTM(128, recurrent_dropout=0.2, return_sequences=True))
model_LSTM_CNN.add(Activation("relu"))
model_LSTM_CNN.add(Dropout(0.2))
model_LSTM_CNN.add(Conv1D(128, 3))
model_LSTM_CNN.add(GlobalMaxPool1D())
model_LSTM_CNN.add(Dropout(0.2))
model_LSTM_CNN.add(Dense(NUM_CLASSES, activation='softmax'))

Скомпилируем модель с параметрами, использованными ранее, и выполним обучение.

In [ ]:
model_LSTM_CNN.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_LSTM_CNN.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 19s 661ms/step - loss: 0.3234 - accuracy: 0.8938 - val_loss: 1.7435 - val_accuracy: 0.7379
Epoch 2/20
25/25 [==============================] - 14s 579ms/step - loss: 0.3048 - accuracy: 0.9002 - val_loss: 1.8151 - val_accuracy: 0.7372


Была выполнена достаточно быстрая остановка обучения, так как уже на второй эпохе качество перестало улучшаться.

In [ ]:
score_LSTM_CNN = model_LSTM_CNN.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_LSTM_CNN[0])
print('Test f1_score:', score_LSTM_CNN[1])

14/14 [==============================] - 2s 152ms/step - loss: 1.7560 - accuracy: 0.7407


Test loss: 1.7559754848480225
Test f1_score: 0.7406864166259766


F1_score получился самым низким среди всех моделей, а loss самым высоким. Таким образом модель LSTM -> CNN справилась хуже остальных архитектур. Возможно, что LSTM, могла создать избыточно длинные или шумные представления, в которых CNN не смогла выделить значимые локальные паттерны из-за потери позиционной информации после рекуррентной обработки.

# Выводы

В работе решалась заклада классификации отзывов.

Было проведено обучение нескольких архитектур нейронных сетей со слоями:

- сверточным,
- SimpleRNN,
- LSTM,
- GRU,
- Bidirectional SimpleRNN,
- CNN -> LSTM,
- LSTM -> CNN.

Для чистоты эксперимента и корректного сравнения моделей использовались идентичные условия: один и тот же оптимизатор (adam), функция потерь (categorical crossentropy), метрика оценки (f1-score), количество эпох 20, размер батча 512, а также идентичная предобработка данных.


Выведем итоговую таблицу.



In [ ]:
pd.DataFrame({'model': ['CNN', 'SimpleRNN', 'LSTM','GRU', 'Bidirectional_SimpleRNN', 'CNN -> LSTM', 'LSTM -> CNN'],
              'test_loss': [score_CNN[0], score_SimpleRNN[0], score_LSTM[0], score_GRU[0], score_BSimpleRNN[0], score_CNN_LSTM[0], score_LSTM_CNN[0]],
              'test_f1_score': [score_CNN[1], score_SimpleRNN[1], score_LSTM[1], score_GRU[1], score_BSimpleRNN[1], score_CNN_LSTM[1], score_LSTM_CNN[1]]})

,model,test_loss,test_f1_score
0,CNN,0.655519,0.776474
1,SimpleRNN,0.710039,0.760634
2,LSTM,0.677596,0.773247
3,GRU,0.690863,0.761807
4,Bidirectional_SimpleRNN,0.738788,0.779994
5,CNN -> LSTM,0.667322,0.772514
6,LSTM -> CNN,1.755975,0.740686


Значения loss изменялись от 1.756 у архитектуры LSTM -> CNN до 0.656 у архитектуры CNN. В целом, учитывая характер данных и несбалансированность классов, можно сказать, что высокое значение loss мы получили только при использовании LSTM -> CNN, все остальные значения получились приемлемыми, но не выше 0.739. Значения f1_score изменялись от 0.761 у SimpleRNN до 0.78 у Bidirectional_SimpleRNN.

Таким образом, наилучший результат показала модель Bidirectional_SimpleRNN, далее CNN, затем LSTM, затем применение совместной архитектуры CNN -> LSTM.